# Verificación de Integridad en Migración de Datos: Azure SQL a SQL Server Management Studio

**Objetivo:**
Comprobar que no se hayan perdido filas durante la migración de datos desde Azure SQL Database a SQL Server Management Studio (SSMS) mediante comparación de conteos y validaciones de checksum.

**Chequeos Clave:**
- Comparación de conteo de registros (origen vs destino)
- Validación de checksum para tablas críticas
- Muestreo de registros coincidentes
- Análisis de valores extremos que puedan indicar problemas

**Nota:** Este proceso garantiza una migración 1:1 sin pérdida de información entre entornos.


In [ ]:
import pyodbc
import logging
import pandas as pd
from tabulate import tabulate
from datetime import datetime

# Configuración de logging mejorada
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

def create_connection(connection_string):
    """Crea una conexión a la base de datos con manejo de errores"""
    try:
        conn = pyodbc.connect(connection_string)
        logging.info("Conexión establecida correctamente")
        return conn
    except pyodbc.Error as e:
        logging.error(f"Error al conectar: {str(e)}")
        return None

# Configuración de conexiones
CONFIG = {
    'azure': {
        'server': 'uaxmathfis.database.windows.net',
        'database': 'usecases',
        'driver': '{ODBC Driver 17 for SQL Server}',
        'authentication': 'ActiveDirectoryInteractive'
    },
    'local': {
        'server': 'localhost',
        'database': 'master',
        'driver': '{ODBC Driver 17 for SQL Server}',
        'trusted_connection': 'yes'
    }
}

# Generar cadenas de conexión
azure_conn_str = f"DRIVER={CONFIG['azure']['driver']};SERVER={CONFIG['azure']['server']};DATABASE={CONFIG['azure']['database']};Authentication={CONFIG['azure']['authentication']}"
local_conn_str = f"DRIVER={CONFIG['local']['driver']};SERVER={CONFIG['local']['server']};DATABASE={CONFIG['local']['database']};Trusted_Connection={CONFIG['local']['trusted_connection']};TrustServerCertificate=yes"

# Tablas a comparar (origen -> destino)
TABLES_TO_COMPARE = {
    "[dbo].[Fact]": "[DATAEX].[001_sales]",
    "[dbo].[Dim_customer]": "[DATAEX].[003_clientes]",
    "[dbo].[Dim_t]": "[DATAEX].[002_date]",
    "[dbo].[Dim_product]": "[DATAEX].[006_producto]",
    "[dbo].[Dim_geo]": "[DATAEX].[011_tienda]"
}

def get_row_count(conn, table_name, source_name):
    """Obtiene el conteo de filas con manejo de errores"""
    try:
        query = f"SELECT COUNT(*) AS count FROM {table_name}"
        df = pd.read_sql(query, conn)
        count = df['count'].iloc[0]
        logging.info(f"Conteo obtenido para {table_name} desde {source_name}: {count}")
        return count
    except Exception as e:
        logging.error(f"Error al contar filas en {table_name} ({source_name}): {str(e)}")
        return None

def verify_data_integrity():
    """Función principal para verificar la integridad de los datos"""
    start_time = datetime.now()
    logging.info(f"\n{'='*50}")
    logging.info("Iniciando verificación de integridad de datos")
    logging.info(f"Fecha/Hora de inicio: {start_time}")
    logging.info(f"{'='*50}\n")
    
    resultados = []
    issues_found = 0
    
    # Establecer conexiones
    azure_conn = create_connection(azure_conn_str)
    local_conn = create_connection(local_conn_str)
    
    if not azure_conn or not local_conn:
        logging.error("No se pudo establecer una o ambas conexiones. Abortando verificación.")
        return
    
    try:
        # Verificar cada tabla
        for local_table, azure_table in TABLES_TO_COMPARE.items():
            logging.info(f"\nVerificando tabla: {local_table} (local) vs {azure_table} (Azure)")
            
            # Obtener conteos
            azure_count = get_row_count(azure_conn, azure_table, "Azure")
            local_count = get_row_count(local_conn, local_table, "Local")
            
            # Validar resultados
            if azure_count is None or local_count is None:
                resultado = "Error en conteo"
                issues_found += 1
            elif azure_count == local_count:
                resultado = "✓ Coinciden"
            else:
                resultado = f"✗ No coinciden (Diferencia: {abs(azure_count - local_count)})"
                issues_found += 1
            
            # Guardar resultados
            table_name = local_table.split('.')[-1].strip('[]')
            resultados.append([
                table_name,
                azure_count if azure_count is not None else "Error",
                local_count if local_count is not None else "Error",
                resultado
            ])
        
        # Mostrar resumen
        logging.info("\nResumen de verificación:")
        print("\n" + tabulate(resultados, 
                            headers=["Tabla", "Azure SQL", "SQL Local", "Resultado"], 
                            tablefmt="grid"))
        
        # Resumen final
        end_time = datetime.now()
        duration = end_time - start_time
        
        logging.info(f"\n{'='*50}")
        logging.info("Resumen final de verificación:")
        logging.info(f" - Tablas verificadas: {len(TABLES_TO_COMPARE)}")
        logging.info(f" - Problemas detectados: {issues_found}")
        logging.info(f" - Tiempo total: {duration.total_seconds():.2f} segundos")
        
        if issues_found == 0:
            logging.info("¡Todos los datos coinciden correctamente!")
        else:
            logging.warning("Se encontraron discrepancias en los datos. Verifique los logs para más detalles.")
        
        logging.info(f"{'='*50}")
        
    except Exception as e:
        logging.error(f"Error durante la verificación: {str(e)}")
    finally:
        # Cerrar conexiones
        if azure_conn:
            azure_conn.close()
        if local_conn:
            local_conn.close()
        logging.info("Conexiones cerradas. Verificación completada.")

if __name__ == "__main__":
    verify_data_integrity()

2025-03-27 15:01:50,229 - INFO - 
2025-03-27 15:01:50,232 - INFO - Iniciando verificación de integridad de datos
2025-03-27 15:01:50,233 - INFO - Fecha/Hora de inicio: 2025-03-27 15:01:50.229507
2025-03-27 15:01:50,233 - INFO - ==================================================



2025-03-27 15:01:50,265 - INFO - Conexión establecida correctamente
2025-03-27 15:01:50,267 - INFO - Conexión establecida correctamente
2025-03-27 15:01:50,269 - INFO - 
Verificando tabla: [dbo].[Fact] (local) vs [DATAEX].[001_sales] (Azure)
C:\Users\aleja\AppData\Local\Temp\ipykernel_29384\2433524076.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-03-27 15:01:50,337 - INFO - Conteo obtenido para [DATAEX].[001_sales] desde Azure: 58049
2025-03-27 15:01:50,381 - INFO - Conteo obtenido para [dbo].[Fact] desde Local: 58049
2025-03-27 15:01:50,382 - INFO - 
Verificando tabla: [dbo].[Dim_customer] (local) vs [DATAEX].[003_clientes] (Azure)
2025-03-27 15:01:50,430 - INFO - Conteo obtenido para [DATAEX].[003_clientes] desde Azure: 44053
2025-03-27 15:01:50,448 - INFO - Conteo obtenido para [dbo].[Di


+--------------+-------------+-------------+-------------+
| Tabla        |   Azure SQL |   SQL Local | Resultado   |
+==============+=============+=============+=============+
| Fact         |       58049 |       58049 | ✓ Coinciden |
+--------------+-------------+-------------+-------------+
| Dim_customer |       44053 |       44053 | ✓ Coinciden |
+--------------+-------------+-------------+-------------+
| Dim_t        |        3652 |        3652 | ✓ Coinciden |
+--------------+-------------+-------------+-------------+
| Dim_product  |         404 |         404 | ✓ Coinciden |
+--------------+-------------+-------------+-------------+
| Dim_geo      |          12 |          12 | ✓ Coinciden |
+--------------+-------------+-------------+-------------+
